In [ ]:
#default_exp data.pipeline

In [ ]:
#export
from local.torch_basics import *
from local.test import *
from local.data.transform import *
from local.notebook.showdoc import show_doc

# Pipeline

> Low-level transform pipelines

The classes here provide functionality for creating a composition of *partially reversible functions*. By "partially reversible" we mean that a transform can be `decode`d, creating a form suitable for display. This is not necessarily identical to the original form (e.g. a transform that changes a byte tensor to a float tensor does not recreate a byte tensor when decoded, since that may lose precision, and a float tensor can be displayed already).

Classes are also provided and for composing transforms, and mapping them over collections. `Pipeline` is a transform which composes several `Transform`, knowing how to decode them or show an encoded item.

## Convenience functions

In [ ]:
#export
def get_func(t, name, *args, **kwargs):
    "Get the `t.name` (potentially partial-ized with `args` and `kwargs`) or `noop` if not defined"
    f = getattr(t, name, noop)
    return f if not (args or kwargs) else partial(f, *args, **kwargs)

This works for any kind of `t` supporting `getattr`, so a class or a module.

In [ ]:
test_eq(get_func(operator, 'neg', 2)(), -2)
test_eq(get_func(operator.neg, '__call__')(2), -2)
test_eq(get_func(list, 'foobar')([2]), [2])
t = get_func(torch, 'zeros', dtype=torch.int64)(5)
test_eq(t.dtype, torch.int64)
a = [2,1]
get_func(list, 'sort')(a)
test_eq(a, [1,2])

## Func -

Transforms are built with multiple-dispatch: a given function can have several methods depending on the type of the object received. This is done directly with the `multimethod` module and type-annotation in `Transform`, but you can also use the following class.

In [ ]:
#export
class Func():
    "Basic wrapper around a `name` with `args` and `kwargs` to call on a given type"
    def __init__(self, name, *args, **kwargs): self.name,self.args,self.kwargs = name,args,kwargs
    def __repr__(self): return f'sig: {self.name}({self.args}, {self.kwargs})'
    def _get(self, t): return get_func(t, self.name, *self.args, **self.kwargs)
    def __call__(self,t): return L(t).mapped(self._get) if is_listy(t) else self._get(t)

You can call the `Func` object on any module name or type, even a list of types. It will return the corresponding function (with a default to `noop` if nothing is found) or list of functions.

In [ ]:
test_eq(Func('sqrt')(math), math.sqrt)
test_eq(Func('sqrt')(torch), torch.sqrt)

@patch
def powx(x:math, a): return math.pow(x,a)
@patch
def powx(x:torch, a): return torch.pow(x,a)
tst = Func('powx',a=2)([math, torch])
test_eq([f.func for f in tst], [math.powx, torch.powx])
for t in tst: test_eq(t.keywords, {'a': 2})

In [ ]:
#export
class _Sig():
    def __getattr__(self,k):
        def _inner(*args, **kwargs): return Func(k, *args, **kwargs)
        return _inner

Sig = _Sig()

In [ ]:
show_doc(Sig, name="Sig")

<h4 id="Sig" class="doc_header"><code>Sig</code><a href="https://github.com/fastai/fastai_dev/tree/master/dev/__main__.py#L4" class="source_link" style="float:right">[source]</a></h4>

> <code>Sig</code>(**\*`args`**, **\*\*`kwargs`**)



`Sig` is just sugar-syntax to create a `Func` object more easily with the syntax `Sig.name(*args, **kwargs)`.

In [ ]:
f = Sig.sqrt()
test_eq(f(math), math.sqrt)
test_eq(f(torch), torch.sqrt)

## Pipeline -

In [ ]:
#export
def compose_tfms(x, tfms, is_enc=True, reverse=False, **kwargs):
    "Apply all `func_nm` attribute of `tfms` on `x`, maybe in `reverse` order"
    if reverse: tfms = reversed(tfms)
    for f in tfms:
        if not is_enc: f = f.decode
        x = f(x, **kwargs)
    return x

In [ ]:
def to_int  (x)  :  return Int(x)
def to_float(x):  return Float(x)
def double(x): return x*2
def half(x)->None: return x/2

In [ ]:
def test_compose(a, b, *fs):
    test_eq_type(compose_tfms(a, tfms=map(Transform,fs)), b)

test_compose(1,   Int(1),   to_int)
test_compose(1,   Float(1), to_int,to_float)
test_compose(1,   Float(2), to_int,to_float,double)
test_compose(2.0, 2.0,      to_int,double,half)

In [ ]:
class A(Transform):
    def encodes(self, x:float):  return Float(x+1)
    def decodes(self, x): return x-1
    
tfms = [A(), Transform(math.sqrt)]
t = compose_tfms(3., tfms=tfms)
test_eq_type(t, Float(2.))
test_eq(compose_tfms(t, tfms=tfms, is_enc=False), 1.)
test_eq(compose_tfms(4., tfms=tfms, reverse=True), 3.)

In [ ]:
tfms = [A(as_item=False), Transform(math.sqrt, as_item=False)]
test_eq(compose_tfms((9,3.), tfms=tfms), (3,2.))

In [ ]:
#export
def batch_to_samples(b, max_n=10):
    "'Transposes' a batch to (at most `max_n`) samples"
    if isinstance(b, Tensor): return b[:max_n]
    else:
        res = L(b).mapped(partial(batch_to_samples,max_n=max_n))
        return L(retain_types(res.zipped(), [b]))

In [ ]:
t = tensor([1,2,3])
test_eq(batch_to_samples([t,t+1], max_n=2), ([1,2],[2,3]))
test_eq(batch_to_samples(tensor([1,2,3]), 10), tensor([1, 2, 3]))
test_eq(batch_to_samples([tensor([1,2,3]), tensor([4,5,6])], 10), [(1, 4), (2, 5), (3, 6)])
test_eq(batch_to_samples([tensor([1,2,3]), tensor([4,5,6])], 2), [(1, 4), (2, 5)])
test_eq(batch_to_samples([tensor([1,2,3]), [tensor([4,5,6]),tensor([7,8,9])]], 10), 
        [(1, (4, 7)), (2, (5, 8)), (3, (6, 9))])
test_eq(batch_to_samples([tensor([1,2,3]), [tensor([4,5,6]),tensor([7,8,9])]], 2), [(1, (4, 7)), (2, (5, 8))])

t = TupleBase(tensor([1,2,3]),TensorBase([2,3,4]))
test_eq_type(batch_to_samples(t)[0][1], TensorBase(2))
test_eq(batch_to_samples(t).mapped(type), [TupleBase]*3)

In [ ]:
#export
def mk_transform(f, as_item=True):
    "Convert function `f` to `Transform` if it isn't already one"
    f = instantiate(f)
    return f if isinstance(f,Transform) else Transform(f, as_item=as_item)

In [ ]:
#export
class Pipeline:
    "A pipeline of composed (for encode/decode) transforms, setup with types"
    def __init__(self, funcs=None, as_item=False, filt=None):
        if isinstance(funcs, Pipeline): funcs = funcs.fs
        self.filt,self.default = filt,None
        self.fs = L(ifnone(funcs,[noop])).mapped(mk_transform).sorted(key='order')
        self.set_as_item(as_item)
        for f in self.fs:
            name = camel2snake(type(f).__name__)
            a = getattr(self,name,None)
            if a is not None: f = L(a)+f
            setattr(self, name, f)

    def set_as_item(self, as_item):
        self.as_item = as_item
        for f in self.fs: f.as_item = as_item

    def setup(self, items=None):
        self.items = items
        tfms,self.fs = self.fs,L()
        for t in tfms: self.add(t,items)

    def add(self,t, items=None):
        t.setup(items)
        self.fs.append(t)

    def __call__(self, o): return compose_tfms(o, tfms=self.fs, filt=self.filt)
    def decode  (self, o): return compose_tfms(o, tfms=self.fs, is_enc=False, reverse=True, filt=self.filt)
    def __repr__(self): return f"Pipeline: {self.fs}"
    def __getitem__(self,i): return self.fs[i]
    def decode_batch(self, b, max_n=10): return batch_to_samples(b, max_n=max_n).mapped(self.decode)
    def __setstate__(self,data): self.__dict__.update(data)
        
    def __getattr__(self,k):
        if k.startswith('_') or k=='fs': raise AttributeError(k)
        res = sum(self.fs.attrgot(k).mapped(L), [])
        if not res: raise AttributeError(k)
        return res[0] if len(res)==1 else res
            
    def show(self, o, ctx=None, **kwargs):
        for f in reversed(self.fs):
            res = self._show(o, ctx, **kwargs)
            if res is not None: return res
            o = f.decode(o, filt=self.filt)
        return self._show(o, ctx, **kwargs)

    def _show(self, o, ctx, **kwargs):
        o1 = [o] if self.as_item or not is_listy(o) else o
        if not all(hasattr(o_, 'show') for o_ in o1): return
        for o_ in o1: ctx = o_.show(ctx=ctx, **kwargs)
        return ifnone(ctx,1)

In [ ]:
add_docs(Pipeline,
         __call__="Compose `__call__` of all `fs` on `o`",
         decode="Compose `decode` of all `fs` on `o`",
         show="Show `o`, a single item from a tuple, decoding as needed",
         add="Add transform `t`",
         decode_batch="`decode` all sample in a the batch `b`",
         set_as_item="Set value of `as_item` for all transforms",
         setup="Call each tfm's `setup` in order")

`Pipeline` is a wrapper for `compose_tfm`. You can pass instances of `Transform` or regular functions in `funcs`, the `Pipeline` will wrap them all in `Transform` (and instantiat them if needed) during the initialization. It handles the transform `setup` by adding them one at a time and calling setup on each, goes through them in order in `__call__` or `decode` and can `show` an object by applying decoding the transforms up until the point it gets an object that knows how to show itself.

In [ ]:
# Empty pipeline is noop
pipe = Pipeline()
test_eq(pipe(1), 1)
pipe.set_as_item(False)
test_eq(pipe((1,)), (1,))
# Check pickle works
assert pickle.loads(pickle.dumps(pipe))

In [ ]:
class IntFloatTfm(Transform):
    def encodes(self, x):  return Int(x)
    def decodes(self, x):  return Float(x)
    foo=1

int_tfm=IntFloatTfm()

def neg(x): return -x
neg_tfm = Transform(neg, neg)

In [ ]:
pipe = Pipeline([neg_tfm, int_tfm])

start = 2.0
t = pipe(start)
test_eq_type(t, Int(-2))
test_eq_type(pipe.decode(t), Float(start))
test_stdout(lambda:pipe.show(t), '-2')

pipe.set_as_item(False)
test_stdout(lambda:pipe.show(pipe((1,2))), '-1\n-2')

Transforms are available as attributes named with the snake_case version of the names of their types. Attributes in transforms can be directly accessed as attributes of the pipeline.

In [ ]:
test_eq(pipe.int_float_tfm, int_tfm)
test_eq(pipe.foo, 1)

pipe = Pipeline([int_tfm, int_tfm])
pipe.int_float_tfm
test_eq(pipe.int_float_tfm[0], int_tfm)
test_eq(pipe.foo, [1,1])

In [ ]:
# Check opposite order
pipe = Pipeline([int_tfm,neg_tfm])
t = pipe(start)
test_eq(t, -2)
test_stdout(lambda:pipe.show(t), '-2')

In [ ]:
class A(Transform):
    def encodes(self, x):  return int(x)
    def decodes(self, x):  return Float(x)
    
pipe = Pipeline([neg_tfm, A])
t = pipe(start)
test_eq_type(t, -2)
test_eq_type(pipe.decode(t), Float(start))
test_stdout(lambda:pipe.show(t), '-2.0')

In [ ]:
s2 = (1,2)
pipe.set_as_item(False)
t = pipe(s2)
test_eq_type(t, (-1,-2))
test_eq_type(pipe.decode(t), (Float(1.),Float(2.)))
test_stdout(lambda:pipe.show(t), '-1.0\n-2.0')

In [ ]:
class B(Transform):
    def encodes(self, x): return x+1
    def decodes(self, x): return x-1

In [ ]:
from PIL import Image

def f1(x:TensorImage): return -x
def f2(x): return Image.open(x).resize((128,128))
def f3(x:Image.Image): return(TensorImage(array(x)))

In [ ]:
pipe = Pipeline([f2,f3,f1])
t = pipe(TEST_IMAGE)
test_eq(type(t), TensorImage)
test_eq(t, -tensor(f3(f2(TEST_IMAGE))))

In [ ]:
pipe = Pipeline([f2,f3])
t = pipe(TEST_IMAGE)
ax = pipe.show(t)

In [ ]:
test_fig_exists(ax)

In [ ]:
#Check filtering is properly applied
add1 = B()
add1.filt = 1
pipe = Pipeline([neg_tfm, A(), add1])
test_eq(pipe(start), -2)
pipe.filt=1
test_eq(pipe(start), -1)
pipe.filt=0
test_eq(pipe(start), -2)
for t in [None, 0, 1]:
    pipe.filt=t
    test_eq(pipe.decode(pipe(start)), start)
    test_stdout(lambda: pipe.show(pipe(start)), "-2.0")

### Methods

In [ ]:
#TODO: method examples

In [ ]:
show_doc(Pipeline.__call__)

In [ ]:
show_doc(Pipeline.decode)

In [ ]:
show_doc(Pipeline.decode_batch)

In [ ]:
pipe.set_as_item(False)
t = tensor([1,2,3])
pipe.filt=1
test_eq(pipe.decode_batch([t,t+1], max_n=2), [(0,-1),(-1,-2)])

In [ ]:
show_doc(Pipeline.setup)

During the setup, the `Pipeline` starts with no transform and adds them one at a time, so that during its setup, each transform gets the items processed up to its point and not after.

In [ ]:
#hide
#Test is below with TfmdList

## TfmdList -

In [ ]:
#export
class TfmdBase(L):
    "Base class for transformed lists"
    def _gets(self, i): return L(self._get(i_) for i_ in mask2idxs(i))
    def subset(self, idxs): return self._new(super()._gets(idxs))
    def decode_at(self, idx): return self.decode(self[idx])
    def show_at(self, idx, **kwargs): return self.show(self[idx], **kwargs)

In [ ]:
#export
class TfmdList(TfmdBase):
    "A `Pipeline` of `tfms` applied to a collection of `items`"
    def __init__(self, items, tfms, do_setup=True, as_item=True, use_list=None, filt=None):
        super().__init__(items, use_list=use_list)
        if isinstance(tfms,TfmdList): tfms = tfms.tfms
        if isinstance(tfms,Pipeline): do_setup=False
        self.tfms = Pipeline(tfms, as_item=as_item, filt=filt)
        if do_setup: self.setup()

    def _new(self, items, *args, **kwargs): return super()._new(items, tfms=self.tfms, do_setup=False, use_list=None, filt=self.filt)
    def _get (self, i): return self.tfms(super()._get(i))
    def __repr__(self): return f"{self.__class__.__name__}: {self.items}\ntfms - {self.tfms.fs}"

    # Delegating to `self.tfms`
    def show(self, o, **kwargs): return self.tfms.show(o, **kwargs)
    def setup(self): self.tfms.setup(self)
    def decode(self, x, **kwargs): return self.tfms.decode(x, **kwargs)
    def __call__(self, x, **kwargs): return self.tfms.__call__(x, **kwargs)

    @property
    def filt(self): return self.tfms.filt
    @filt.setter
    def filt(self,v): self.tfms.filt = v

In [ ]:
add_docs(TfmdList,
         setup="Transform setup with self",
         decode="From `Pipeline",
         show="From `Pipeline",
         decode_at="Decoded item at `idx`",
         show_at="Show item at `idx`",
         subset="New `TfmdList` that only includes items at `idxs`")

A `TfmdList` combines a collection of object with a `Pipeline`. `tfms` can either be a `Pipeline` or a list of transforms, in which case, it will wrap them in a `Pipeline`. `use_list` is passed along to `L` with the `items`, `as_item` and `filt` are passed to each transform of the `Pipeline`. `do_setup` indicates if the `Pipeline.setup` method should be called during initialization.

In [ ]:
tl = TfmdList([1.,2.,3.], [neg_tfm, int_tfm])
t = tl[1]
test_eq_type(t, Int(-2))
test_eq(tl.decode_at(1), 2)
test_eq_type(tl.decode(t), Float(2.0))
test_stdout(lambda: tl.show_at(2), '-3')
tl

In [ ]:
t = tl[1]
test_eq_type(t, Int(-2))
test_eq(tl.decode_at(1), 2)
test_eq_type(tl.decode(t), Float(2.0))
test_stdout(lambda: tl.show_at(2), '-3')
tl

In [ ]:
p2 = tl.subset([0,2])
test_eq(p2, [-1,-3])
test_eq(map(type, p2), (Int,Int))

In [ ]:
df = pd.DataFrame(dict(a=[1,2,3],b=[2,3,4]))
tl = TfmdList(df, lambda o: o.a)
test_eq(tl[1,2], [2,3])
p2 = tl.subset([1,2])
test_eq(p2, [2,3])

Here's how we can use `TfmdList.setup` to implement a simple category list, getting labels from a mock file list:

In [ ]:
class _Cat(Transform):
    order = 1
    def __init__(self, subset_idx=None): self.subset_idx = subset_idx
    def encodes(self, o):  return int(self.o2i[o])
    def decodes(self, o):  return Str(self.vocab[o])
    def setups(self, items): 
        if self.subset_idx is not None: items = items.subset(self.subset_idx)
        self.vocab,self.o2i = uniqueify(L(items), sort=True, bidir=True)

def _lbl(o):  return Str(o.split('_')[0])

test_fns = ['dog_0.jpg','cat_0.jpg','cat_2.jpg','cat_1.jpg','dog_1.jpg']
tcat = _Cat()
# Check that tfms are sorted by `order`
tl = TfmdList(test_fns, [tcat,_lbl])

test_eq(tcat.vocab, ['cat','dog'])

In [ ]:
test_eq(tl, (1,0,0,0,1))
t = L(tl)
test_eq(t, [1,0,0,0,1])
test_eq(tl[-1], 1)
test_eq(tl[0,1], (1,0))
test_eq([tl.decode(o) for o in t], ('dog','cat','cat','cat','dog'))
test_stdout(lambda:tl.show_at(0), "dog")
tl

In [ ]:
class B(Transform):
    def __init__(self):   self.a = 2
    def encodes(self, x): return x+self.a
    def decodes(self, x): return x-self.a
    def setups(self, items): self.a = tensor(items).float().mean().item()

tl1 = TfmdList([1,2,3,4], B())
test_eq(tl1.tfms[0].a, 2.5)

In [ ]:
#hide
#Check filtering is properly applied
tl1 = TfmdList([1.,2.,3.,4.], [neg_tfm, A(), add1])
test_eq(tl1[2], -3)
tl1.filt=1
test_eq(tl1[2], -2)
test_eq(tl1[[2,2]], [-2,-2])
tl1.filt=0
test_eq(tl1[2], -3)
for t in [None, 0, 1]:
    tl1.filt=t
    test_eq(tl1.decode(tl1[1]), 2)
    test_eq(tl1.decode_at(1), 2)
    test_stdout(lambda: tl1.show_at(1), "-2.0")

### Methods

In [ ]:
show_doc(TfmdList.decode_at)

In [ ]:
test_eq(tl.decode_at(1),tl.decode(tl[1]))

In [ ]:
show_doc(TfmdList.show_at)

In [ ]:
test_stdout(lambda: tl.show_at(1), 'cat')

In [ ]:
show_doc(TfmdList.subset)

## TfmdDS -

In [ ]:
#export
@docs
class TfmdDS(TfmdBase):
    "A dataset that creates a tuple from each `tfms`, passed thru `ds_tfms`"
    def __init__(self, items, tfms=None, do_setup=True, use_list=None, filt=None):
        super().__init__(items, use_list=use_list)
        if tfms is None: tms = [None]
        self.tls = [TfmdList(items, t, do_setup=do_setup, filt=filt, use_list=use_list) for t in L(tfms)]

    def _get(self, it): return tuple(tl._get(it) for tl in self.tls)
    def __repr__(self): return coll_repr(self)
    def decode(self, o): return tuple(it.decode(o_) for o_,it in zip(o,self.tls))
    def show(self, o, ctx=None, **kwargs):
        for o_,it in zip(o,self.tls): ctx = it.show(o_, ctx=ctx, **kwargs)
        return ctx

    @property
    def filt(self): return self.tls[0].filt
    @filt.setter
    def filt(self,v):
        for tl in self.tls: tl.filt = v

    _docs=dict(
        decode="Compose `decode` of all `tuple_tfms` then all `tfms` on `i`",
        show="Show item `o` in `ctx`")

A `TfmdDS` creates a tuple from `items` (typically input,target) by applying each list of `Transform` (or `Pipeline`) in `tfms` to them. Note that if `tfms` contains only one list of `tfms`, the items given by `TfmdDS` will be tuples of one element.

In [ ]:
items = [1,2,3,4]
tds = TfmdDS(items, [[neg_tfm,int_tfm]])
test_eq(*tds[0], -1)
test_eq(tds[0,1,2], [(-1,),(-2,),(-3,)])

In [ ]:
class A(Transform):
    def encodes(self, o): return (o-self.m)/self.s
    def decodes(self, o): return (o*self.s)+self.m
    def setups(self, items):
        its = tensor(items).float()
        self.m,self.s = its.mean(),its.std()

In [ ]:
items = [1,2,3,4]
nrm = A()
tds = TfmdDS(items, [[neg_tfm,int_tfm], [neg_tfm,nrm]])

x,y = zip(*tds)
test_close(tensor(y).mean(), 0)
test_close(tensor(y).std(), 1)
test_eq(x, (-1,-2,-3,-4,))
test_eq(nrm.m, -2.5)
test_stdout(lambda:tds.show_at(1), '-2')
#TODO: do something here
# test_eq(tds.m, tds.type_tfms[1].fs[1].m)
# test_eq(tds.s, tds.type_tfms[1].fs[1].s)
#Attributes are set to all parents progressively
# test_eq(tds.tls[1].m, tds.type_tfms[1].fs[1].m)
# test_eq(tds.tls[1].s, tds.type_tfms[1].fs[1].s)

In [ ]:
#hide
#Check filtering is properly applied
class B(Transform):
    def encodes(self, x)->None:  return int(x+1)
    def decodes(self, x):        return Int(x-1)

add1 = B()
add1.filt = 1

tds = TfmdDS(items, [neg_tfm, [neg_tfm,int_tfm,add1]])
test_eq(tds[1], [-2,-2])
tds.filt=1
test_eq(tds.filt, 1)
test_eq(tds[1], [-2,-1])
test_eq(tds[[1,1]], [[-2,-1], [-2,-1]])
tds.filt=0
test_eq(tds[1], [-2,-2])
for t in [None, 0, 1]:
    tds.filt=t
    test_eq(tds.decode(tds[1]), [2,2])
    test_eq(tds.decode_at(1), [2,2])
    test_stdout(lambda: tds.show_at(1), "-2")

### Methods

In [ ]:
items = [1,2,3,4]
tds = TfmdDS(items, [[neg_tfm,int_tfm]])

In [ ]:
show_doc(TfmdDS.decode)

In [ ]:
test_eq(*tds[0], -1)
test_eq(*tds.decode((-1,)), 1)

In [ ]:
show_doc(TfmdDS.decode_at)

In [ ]:
test_eq(tds.decode_at(0), (1,))

In [ ]:
show_doc(TfmdDS.show)

In [ ]:
test_stdout(lambda:tds.show(tds[1]), '-2')

In [ ]:
show_doc(TfmdDS.show_at)

In [ ]:
test_stdout(lambda:tds.show_at(1), '-2')

## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)